# Вебинар 6. Двухуровневые модели рекомендаций


### Зачем 2 уровня?
- Классические модели классификации (lightgbm) зачастую работают лучше, чем рекоммендательные модели (als, lightfm)
- Данных много, предсказаний много (# items * # users) --> с таким объемом lightgbm не справляется
- Но рекомендательные модели справляются!

Отбираем top-N (200) *кандидатов* с помощью простой модели (als) --> переранжируем их сложной моделью (lightgbm)
и выберем top-k (10).

---

### Применение в бизнесе

Если вы еще не прочитали [статью](https://habr.com/ru/company/hh/blog/347276/) о рекомендательных системах и поиске в hh.ru, то обязательно прочитайте

- 2-ух уровневая система применяется во многих компаниях
- Зачастую уровней > 2
- Идем от более простых эвристик/моделей к более сложным
- Фичи из моделей первого уровня (embeddings, biases из ALS) можно использовать в последующих моделях

Также решения на основе 2-ух уровневых рекомендаций заняли все топ-10 мест в соревновании X5 Retail hero. 

- [Презентация](https://github.com/aprotopopov/retailhero_recommender/blob/master/slides/retailhero_recommender.pdf) и [Код](https://github.com/aprotopopov/retailhero_recommender) решения 2-ого места
- [Код](https://vk.com/away.php?utf=1&to=https%3A%2F%2Fgithub.com%2Fmike-chesnokov%2Fx5_retailhero_2020_recs) решения 9-ого места

### Как отбирать кандидатов?

Вариантов множество. Тут нам поможет *MainRecommender*. Пока в нем реализованы далеко не все возможные способы генерации кандидатов

- Генерируем топ-k кандидатов
- Качество кандидатов измеряем через **recall@k**
- recall@k показывает какую долю из купленных товаров мы смогли выявить (рекомендовать) нашей моделью

----

Pipline:
1. Рекомендуем 50 кандидатов среди товаров классическими методами
2. Оцениваем recall@k нашу кандидатную выдачу (выдача моделями 1-го уровня)
3. Получаем user-item датасет по кандидатным рекомендациям
4. Для такого датасета проставляем target купил/не купил товар по истории взаимодействий
5. На этом датасете строим lightGBM, предсказывающий купит или не купит пользователь данный товар 

# Практическая часть

Код для src, utils, metrics вы можете скачать с [этого](https://github.com/geangohn/recsys-tutorial) github репозитория

In [37]:
!pip install implicit

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [39]:
data = pd.read_csv('retail_train.csv')
item_features = pd.read_csv('product.csv')
user_features = pd.read_csv('hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)


# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [40]:
n_items_before = data_train_lvl_1['item_id'].nunique()

data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features, take_n_popular=5000)

n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

/home/ivan/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/mnt/c/Users/Zalut/Documents/GitHub/GBEdu/Recomendation_sisten/src/utils.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))


Decreased # items from 83685 to 5145


In [41]:
recommender = MainRecommender(data_train_lvl_1)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5145 [00:00<?, ?it/s]

In [42]:
recommender

### Варианты, как получить кандидатов

Можно потом все эти варианты соединить в один

(!) Если модель рекомендует < N товаров, то рекомендации дополняются топ-популярными товарами до N

In [43]:
recommender.get_als_recommendations(2375, N=5)

[925862, 963727, 1128244, 850925, 939900]

In [44]:
recommender.get_own_recommendations(2375, N=5)

[1085983, 910439, 907099, 1005935, 1027642]

In [45]:
recommender.get_similar_items_recommendation(2375, N=5)

[1038325, 896613, 1134678, 948650, 1003616]

In [46]:
recommender.get_similar_users_recommendation(2375, N=5)

[1087411, 1035207, 1056212, 955858, 1134296]

### Измеряем recall@k

Это будет в ДЗ: 

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 50 кандидатов (k=50)
- Качество измеряем на data_val_lvl_1: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  
C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?


In [47]:
result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


In [48]:
N = 50
users_train = data_train_lvl_1['user_id'].tolist()
users_valid = result_lvl_1['user_id'].tolist()
new_users = list(set(users_valid) - set(users_train))
users_list = list(set(users_valid) & set(users_train))

cols = ['top_popular', 'als_recommendations', 'own_recommendations', 'similar_items_recommendation',
        'similar_users_recommendation']
df = pd.DataFrame(index=users_valid, columns=cols)

overall_top_purchases = recommender.overall_top_purchases[:N]

for user in users_valid:
    for col in cols:
        df.loc[user, col] = overall_top_purchases 
        
for user in users_list:
    df.loc[user, 'top_popular'] = overall_top_purchases
    df.loc[user, 'als_recommendations'] = recommender.get_als_recommendations(user, N)
    df.loc[user, 'own_recommendations'] = recommender.get_own_recommendations(user, N)
    df.loc[user, 'similar_items_recommendation'] = recommender.get_similar_items_recommendation(user, N)    
    df.loc[user, 'similar_users_recommendation'] = recommender.get_similar_users_recommendation(user, N)  
            
df = df.reset_index()
df.columns = ['user_id'] + cols
result_lvl_1 = result_lvl_1.merge(df, on='user_id', how='left')
result_lvl_1.head(2)

,user_id,actual,top_popular,als_recommendations,own_recommendations,similar_items_recommendation,similar_users_recommendation
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[823704, 5569374, 1070702, 1041796, 898068, 10...","[962615, 888543, 5570513, 10356272, 1015386, 7...","[856942, 1049998, 9297615, 5577022, 1082269, 8...","[904777, 13382307, 9297615, 5577022, 9420210, ...","[1029125, 1135983, 9831557, 1015781, 983267, 9..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870...","[823704, 5569374, 1070702, 1041796, 898068, 10...","[972665, 5581464, 985579, 1028950, 10456568, 9...","[1076580, 7433029, 1103898, 831125, 5567582, 1...","[989150, 1026058, 12524365, 885686, 1082814, 1...","[1076580, 1125601, 869252, 979877, 9881593, 10..."


In [49]:
result_lvl_1['own_recommendtions_top_popular'] = result_lvl_1['own_recommendations'] + df['top_popular']
len(result_lvl_1.own_recommendtions_top_popular[0])

100

In [50]:
for col in result_lvl_1.columns[2:]:    
    recall = result_lvl_1.apply(lambda row: recall_at_k(row[col], row['actual'], N), axis=1).mean()
    print(f'recall_at_{len(result_lvl_1[col][0])} {col}: {round(recall, 4)}')

recall_at_50 top_popular: 0.0104
recall_at_50 als_recommendations: 0.0266
recall_at_50 own_recommendations: 0.0565
recall_at_50 similar_items_recommendation: 0.0159
recall_at_50 similar_users_recommendation: 0.0067
recall_at_100 own_recommendtions_top_popular: 0.0565


**Ответ:**
- А) Совмещение own recommendtions + top-popular recall лучше не стал

In [51]:
k = [20, 50, 100, 200, 500]
cols = ['own_recommendations_' + str(i) for i in k]
df = pd.DataFrame(index=users_valid, columns=cols)

for i, N in enumerate(k):
    overall_top_purchases = recommender.overall_top_purchases[:N]
    for user in new_users:        
        df.loc[user, cols[i]] = overall_top_purchases
        
    for user in users_list:
        df.loc[user, cols[i]] = recommender.get_own_recommendations(user, N)
        
df = df.reset_index()
df.columns = ['user_id'] + cols
result_lvl_1 = result_lvl_1.merge(df, on='user_id', how='left')

In [52]:
for i, N in enumerate(k):   
    recall = result_lvl_1.apply(lambda row: recall_at_k(row[cols[i]], row['actual'], N), axis=1).mean()
    print(f'recall_at_{N} {cols[i]}: {round(recall, 4)}')

recall_at_20 own_recommendations_20: 0.0375
recall_at_50 own_recommendations_50: 0.0565
recall_at_100 own_recommendations_100: 0.0732
recall_at_200 own_recommendations_200: 0.09
recall_at_500 own_recommendations_500: 0.1116


**Ответ:** 
- В)* При увеличении количества кандидатов recall@k увеличивается 
- C)* Все очень индивидуально. Зависит от рекомендуемых item, категории users, способа рекомендаций.

### Обучаем модель 2-ого уровня на выбранных кандидатах

- Обучаем на data_train_lvl_2
- Обучаем *только* на выбранных кандидатах
- Я *для примера* сгенерирую топ-50 кадидиатов через get_own_recommendations
- (!) Если юзер купил < 50 товаров, то get_own_recommendations дополнит рекоммендации топ-популярными

In [53]:
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 

In [54]:
users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
users_lvl_2.columns = ['user_id']

# Пока только warm start
train_users = data_train_lvl_1['user_id'].unique()
users_lvl_2 = users_lvl_2[users_lvl_2['user_id'].isin(train_users)]

users_lvl_2['candidates'] = users_lvl_2['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=50))

In [55]:
users_lvl_2.head(2)

,user_id,candidates
0,2070,"[917033, 1107824, 944588, 1008814, 869868, 113..."
1,2021,"[950935, 1119454, 835578, 863762, 1051489, 102..."


In [56]:
s = users_lvl_2.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'item_id'

users_lvl_2 = users_lvl_2.drop('candidates', axis=1).join(s)
users_lvl_2['flag'] = 1

users_lvl_2.head(4)

,user_id,item_id,flag
0,2070,917033,1
0,2070,1107824,1
0,2070,944588,1
0,2070,1008814,1


In [57]:
users_lvl_2.shape[0]

107650

In [58]:
users_lvl_2['user_id'].nunique()

2153

In [59]:
targets_lvl_2 = data_train_lvl_2[['user_id', 'item_id']].copy()
targets_lvl_2['target'] = 1  # тут только покупки 

targets_lvl_2 = users_lvl_2.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')

targets_lvl_2['target'].fillna(0, inplace= True)
targets_lvl_2.drop('flag', axis=1, inplace=True)

In [60]:
targets_lvl_2.head(2)

,user_id,item_id,target
0,2070,917033,0.0
1,2070,1107824,0.0


(!) На каждого юзера 50 item_id-кандидатов

In [61]:
targets_lvl_2['target'].mean()

0.10879431986698422

Слайд из [презентации](https://github.com/aprotopopov/retailhero_recommender/blob/master/slides/retailhero_recommender.pdf) решения 2-ого места X5 Retail Hero

- Пока для простоты обучения выберем LightGBM c loss = binary. Это классическая бинарная классификация
- Это пример *без* генерации фич

In [62]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [63]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [64]:
targets_lvl_2 = targets_lvl_2.merge(item_features, on='item_id', how='left')
targets_lvl_2 = targets_lvl_2.merge(user_features, on='user_id', how='left')

targets_lvl_2.head(2)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,2070,917033,0.0,103,GROCERY,National,SOFT DRINKS,SOFT DRINKS 12/18&15PK CAN CAR,12 OZ,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
1,2070,1107824,0.0,544,GROCERY,National,WAREHOUSE SNACKS,SNACK MIX,7.75 OZ,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown


**Фичи user_id:**
    - Средний чек
    - Средняя сумма покупки 1 товара в каждой категории
    - Кол-во покупок в каждой категории
    - Частотность покупок раз/месяц
    - Долю покупок в выходные
    - Долю покупок утром/днем/вечером

**Фичи item_id**:
    - Кол-во покупок в неделю
    - Среднее ол-во покупок 1 товара в категории в неделю
    - (Кол-во покупок в неделю) / (Среднее кол-во покупок 1 товара в категории в неделю)
    - Цена (Можно посчитать из retil_train.csv)
    - Цена / Средняя цена товара в категории
    
**Фичи пары user_id - item_id**
    - (Средняя сумма покупки 1 товара в каждой категории (берем категорию item_id)) - (Цена item_id)
    - (Кол-во покупок юзером конкретной категории в неделю) - (Среднее кол-во покупок всеми юзерами конкретной категории в неделю)
    - (Кол-во покупок юзером конкретной категории в неделю) / (Среднее кол-во покупок всеми юзерами конкретной категории в неделю)

In [65]:
X_train = targets_lvl_2.drop('target', axis=1)
y_train = targets_lvl_2[['target']]

In [66]:
cat_feats = X_train.columns[2:].tolist()
X_train[cat_feats] = X_train[cat_feats].astype('category')

cat_feats

['manufacturer',
 'department',
 'brand',
 'commodity_desc',
 'sub_commodity_desc',
 'curr_size_of_product',
 'age_desc',
 'marital_status_code',
 'income_desc',
 'homeowner_desc',
 'hh_comp_desc',
 'household_size_desc',
 'kid_category_desc']

In [67]:
lgb = LGBMClassifier(objective='binary', max_depth=7, categorical_column=cat_feats)
lgb.fit(X_train, y_train)

train_preds = lgb.predict(X_train)

/home/ivan/.local/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ivan/.local/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ivan/.local/lib/python3.8/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


In [68]:
len(X_train)

111265

In [69]:
len(train_preds)

111265

Берем топ-k предсказаний, ранжированных по вероятности, для каждого юзера

In [70]:
train_preds.mean()

0.015054150002246887

In [71]:
len(train_preds)

111265

In [72]:
X_train['preds'] = train_preds

In [73]:
X_train.head()

,user_id,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,preds
0,2070,917033,103,GROCERY,National,SOFT DRINKS,SOFT DRINKS 12/18&15PK CAN CAR,12 OZ,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,0.0
1,2070,1107824,544,GROCERY,National,WAREHOUSE SNACKS,SNACK MIX,7.75 OZ,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,0.0
2,2070,944588,1094,MEAT-PCKGD,National,LUNCHMEAT,HAM,12 OZ,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,0.0
3,2070,1008814,103,GROCERY,National,SOFT DRINKS,SOFT DRINKS 12/18&15PK CAN CAR,12 OZ,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,0.0
4,2070,869868,1230,GROCERY,National,SOUP,RTS SOUP: CHUNKY/HOMESTYLE ET,15 OZ,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,0.0


In [74]:
X_train[X_train.preds == 1]['user_id'].nunique()

358

In [75]:
X_train['user_id'].nunique()

2153

In [76]:
# val_preds = lgb.predict(X_val), где X_val - датасет подготовленный по аналогии с data_train_lvl_2